In [10]:
import numpy as np
import pandas as pd
import requests
import re
import ftplib
import xarray as xr
import os
import appdirs
import time
from datetime import datetime
from io import StringIO
import matplotlib.pyplot as plt
from dateutil import relativedelta

In [11]:
pre_name = 'Baseline_ilt_extended'
plname = 'Trend_' + pre_name
tag = ''
predictors = pd.read_csv('/home/poyraden/MLR_Uccle/Files/Extended_ilt.csv')
# For DeBilt
predictors = predictors.loc['1992-11-01':'2018-12-01']
predictors

,Unnamed: 0,linear_pre,linear_post,post_const,pre_const,gap_const,enso,qboA,qboB,solar
200,1985-09,-1.133494,0.0,0.0,1.0,0.0,-0.535,0.519112,-1.026784,-1.088319
201,1985-10,-1.125280,0.0,0.0,1.0,0.0,-0.148,0.149196,-1.143714,-1.002959


In [12]:
from LOTUS_regression.regression import mzm_regression


uccle = pd.read_csv('/home/poyraden/MLR_Uccle/Files/DeBilt_1km_monthlymean_deseas.csv')
print(list(uccle))
#dates = pd.date_range(start='1992-11', end='2018-12', freq = 'MS')

uccle.rename(columns={'Unnamed: 0':'date'}, inplace=True)
#uccle['date'] =  dates
pd.to_datetime(uccle['date'], format='%Y-%m')
uccle.set_index('date', inplace=True)

print(list(uccle))
alt = [''] * 36
alt_ds = [''] * 36

# uc = pd.DataFrame()
# uct = pd.DataFrame()
uc = {}
uct = {}
ucm = {}

regression_output = [0] * 36
uX = [0] * 36
uY = [0] * 36
param_list = [0] * 36
error_list = [0] * 36
ut = [0] * 36

trend_pre = [0] * 36
trend_pre_err = [0] * 36
trend_post = [0] * 36
trend_post_err = [0] * 36

mY = []

uct_pre = {}
uct_post = {}

mean_pre = [0]*36
mean_post = [0]*36

for i in range(36):
    mY.append(i)
    alt_ds[i] = str(i) + 'km_ds'
    alt[i] = str(i) + 'km'

    uc[i] = uccle
    # ucm neede to calculate the monthly means, >0 is neded to eliminate -9999 values
    # ucm[i] = uccle[uccle[alt[i]]>0]
    #uc[alt[i]] = uccle[alt[i]]

    #uct[i] = uc[i].loc['1992-11-15':'20178-12-15']
    uct[i] = uc[i]

    # uct_pre[i] = ucm[i].loc['1977-02-01':'1996-12-01']
    # uct_post[i] = ucm[i].loc['2000-02-01':'2017-06-01']

    predictors, uct[i] = pd.DataFrame.align(predictors, uct[i], axis=0)

    uY[i] = uct[i][alt_ds[i]].values

    # mean_pre[i] = np.nanmean(uct_pre[i][alt[i]].values)
    # mean_post[i] = np.nanmean(uct_post[i][alt[i]].values)

    uX[i] = predictors.values
    
    print(i, uX[i])

    regression_output[i] = mzm_regression(uX[i], uY[i])
    param_list[i] = dict(zip(list(predictors), regression_output[i]['gls_results'].params))
    error_list[i] = dict(zip(list(predictors), regression_output[i]['gls_results'].bse))

    ut[i] = uct[i].index
    ptitle = str(alt[i])
    pname =str(alt[i])

    if (i == 24): print(i, len(ut[i]), len(uY[i]), len(regression_output[i]))


    trend_pre[i] = param_list[i]['linear_pre']
    trend_pre_err[i] = error_list[i]['linear_pre']
    trend_post[i] = param_list[i]['linear_post']
    trend_post_err[i] = error_list[i]['linear_post']

    print('pre', trend_pre[i], 'post', trend_post[i])


    # for % in decade for relative montly anamoly

    trend_pre[i] = trend_pre[i] * 100
    trend_pre_err[i] = 2 * trend_pre_err[i] * 100
    trend_post[i] = trend_post[i] * 100
    trend_post_err[i] = 2 * trend_post_err[i] *100

['date', '0km_ds', '1km_ds', '2km_ds', '3km_ds', '4km_ds', '5km_ds', '6km_ds', '7km_ds', '8km_ds', '9km_ds', '10km_ds', '11km_ds', '12km_ds', '13km_ds', '14km_ds', '15km_ds', '16km_ds', '17km_ds', '18km_ds', '19km_ds', '20km_ds', '21km_ds', '22km_ds', '23km_ds', '24km_ds', '25km_ds', '26km_ds', '27km_ds', '28km_ds', '29km_ds', '30km_ds', '31km_ds', '32km_ds', '33km_ds', '34km_ds', '35km_ds']
['0km_ds', '1km_ds', '2km_ds', '3km_ds', '4km_ds', '5km_ds', '6km_ds', '7km_ds', '8km_ds', '9km_ds', '10km_ds', '11km_ds', '12km_ds', '13km_ds', '14km_ds', '15km_ds', '16km_ds', '17km_ds', '18km_ds', '19km_ds', '20km_ds', '21km_ds', '22km_ds', '23km_ds', '24km_ds', '25km_ds', '26km_ds', '27km_ds', '28km_ds', '29km_ds', '30km_ds', '31km_ds', '32km_ds', '33km_ds', '34km_ds', '35km_ds']
0 [['1985-09' -1.1334935008932419 0.0 ..., 0.5191123024038141
  -1.0267837645863476 -1.0883187133704455]
 ['1985-10' -1.125279779872277 0.0 ..., 0.14919635679525753
  -1.143713501737061 -1.0029589769547185]
 [nan nan n

TypeError: ufunc 'isfinite' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [9]:
predictors = pd.read_csv('/home/poyraden/MLR_Uccle/Files/1km_monthlymean_all_relative.csv')
predictors


,date,0km,1km,2km,3km,4km,5km,6km,7km,8km,...,26km_ds,27km_ds,28km_ds,29km_ds,30km_ds,31km_ds,32km_ds,33km_ds,34km_ds,35km_ds
0,1969-01-01,2.912,2.989,3.170,3.050,2.810,2.582,2.356,1.948,1.794,...,0.176023,0.150776,0.143983,0.164397,0.279994,0.490394,0.616999,0.726390,0.684176,NaN
1,1969-02-01,2.508,1.944,1.801,1.997,2.016,1.823,2.131,2.838,4.030,...,0.134114,0.164925,0.228390,0.269485,0.244483,0.311861,0.397392,0.462365,0.463972,0.441237
2,1969-03-01,1.832,2.352,3.002,3.077,2.906,2.517,2.097,1.852,1.843,...,-0.006758,-0.002138,-0.035659,-0.016093,0.018245,0.051473,0.096090,0.156548,0.142154,0.249384
3,1969-04-01,2.000,3.164,3.738,3.569,3.055,2.574,2.121,1.859,1.617,...,0.012754,0.016034,0.031780,0.041640,0.068683,0.043845,0.035349,0.064540,0.088078,0.100840
4,1969-05-01,1.378,2.449,3.464,3.617,3.473,3.258,2.857,2.779,3.667,...,0.013661,0.030910,0.054326,0.065788,0.077930,0.091798,0.147521,0.177979,0.242554,0.211965
5,1969-06-01,1.661,2.390,3.583,3.797,3.603,3.339,2.984,2.477,2.609,...,-0.002149,0.003326,0.051234,0.078168,0.113462,0.133000,0.141586,0.193327,0.186884,0.247376
6,1969-07-01,1.949,3.318,4.003,3.954,3.261,3.229,2.818,2.855,3.273,...,0.040038,0.024378,0.023267,-0.015241,-0.000063,-0.002592,-0.012202,0.007112,0.024068,0.026530
7,1969-08-01,2.118,3.415,3.730,3.356,3.114,2.910,2.898,2.173,2.051,...,0.064424,0.052921,0.027406,0.031864,0.059729,0.083590,0.129085,0.155546,0.146483,0.135923
8,1969-09-01,-9999.000,3.735,3.701,3.793,3.482,3.193,2.807,2.298,2.019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1969-10-01,-9999.000,2.760,2.383,2.861,2.782,2.398,2.005,1.724,1.480,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
